In [39]:
import os, os.path
import numpy as np
import pandas as pd
import setup_analysis as sa
import support_functions as sf
import importlib
importlib.reload(sa)

importlib.reload(sf)


<module 'support_functions' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/support_functions.py'>

In [110]:
class AFOLU:
    
    def __init__(self, attributes):
        
        self.model_attributes = attributes
        self.required_subsectors = self.get_required_subsectors()
        self.required_variables = self.get_required_variables()
        
        # specify basic field values
        self.field_gdp = self.model_attributes.build_varlist("Economy", "Gross Domestic Product")[0]
        self.field_pop_total = "population_total"
        
        ##  get lists of fields 
        
        # SE
        self.fields_pop = self.model_attributes.build_varlist("General", "Population")
        self.fields_va = self.model_attributes.build_varlist("Economy", "Value Added")
        # LVST
        self.fields_lvst_elasticities = self.model_attributes.build_varlist("Livestock", "Elasticity of Livestock Demand to GDP per Capita")
        
        
    
    def get_required_subsectors(self):
        subsectors = list(sf.subset_df(sa.model_attributes.dict_attributes["abbreviation_subsector"].table, {"sector": ["AFOLU"]})["subsector"])
        subsectors += ["Economy", "General"]
        return subsectors
        
    def get_required_variables(self, build_df_q = False):
        vars_out = []
        subsectors_out = []
        
        for subsector in self.required_subsectors:
            vars_subsector = self.model_attributes.build_varlist(subsector)
            vars_out += vars_subsector
            if build_df_q:
                subsectors_out += [subsector for x in vars_subsector]
        if build_df_q:
            vars_out = pd.DataFrame({"subsector": subsectors_out, "variable": vars_out}).sort_values(by = ["subsector", "variable"]).reset_index(drop = True)
            
        return vars_out
    
    def check_df_fields(self, df_afolu_trajectories):
        # check for required variables
        if not set(self.required_variables).issubset(df_afolu_trajectories.columns):
            set_missing = list(set(self.required_variables) - set(df_afolu_trajectories.columns))
            set_missing = sf.format_print_list(set_missing)
            print(set_missing)
            print("HERE!")
            raise KeyError(f"AFOLU projection cannot proceed: The fields {set_missing} are missing.")
            
            
    def project(self, df_afolu_trajectories):
        
        
        self.check_df_fields(df_afolu_trajectories)
        
        
        
        
        # add to data frame
        df_afolu_trajectories[field_pop_total] = df_afolu_trajectories[sa.model_attributes.build_varlist("General", "Population")].sum(axis = 1).astype(int)
        
        # calculate some vectors
        gdp_per_capita = np.array(df_afolu_trajectories[field_gdp])/np.array(df_afolu_trajectories[field_pop_total])
        gdp_per_capita_rates = gdp_per_capita[1:]/gdp_per_capita[0:-1] - 1
        
        
        
        
        
        print(gdp_per_capita_rates)
    
        

In [115]:
importlib.reload(sa)
model_afolu = AFOLU(sa.model_attributes)
model_afolu.get_required_variables(True).to_csv(os.path.join(sa.dir_out, "tmp_vars_for_afolu2.csv"), encoding = "UTF-8", index = None)

if False:
    df_fake_data = pd.read_csv(os.path.join(sa.dir_out, "fake_data_afolu.csv"))
    df_fake_data[sa.model_attributes.build_varlist("Economy", "Gross Domestic Product")[0]] = df_fake_data[sa.model_attributes.build_varlist("Economy", "Value Added")].sum(axis = 1)

model_afolu.project(df_fake_data)
sa.model_attributes.build_varlist("Livestock", "Elasticity of Livestock Demand to GDP per Capita")

[0.03171666 0.02630725 0.01726744 0.01966599 0.0189801  0.02132296
 0.0226857  0.02521598 0.02753739 0.02787171 0.02819933 0.02852765
 0.02885411 0.02917832 0.029501   0.02982061 0.03013063 0.03043763
 0.03072904 0.03101421 0.03129478 0.03157127 0.03184636 0.03211655
 0.03238272 0.03264754 0.03290862 0.03317058 0.03343909 0.03370869
 0.03397503 0.034239   0.03449933 0.03475635 0.0350111 ]


['elasticity_cattle_dairy_demand_to_gdppc',
 'elasticity_cattle_nondairy_demand_to_gdppc',
 'elasticity_sheep_demand_to_gdppc',
 'elasticity_chickens_demand_to_gdppc',
 'elasticity_pigs_demand_to_gdppc']

In [100]:
df_fake_data = pd.read_csv(os.path.join(sa.dir_out, "fake_data_afolu.csv"))
# add some vars
df_fake_data[sa.model_attributes.build_varlist("Economy", "Gross Domestic Product")[0]] = df_fake_data[sa.model_attributes.build_varlist("Economy", "Value Added")].sum(axis = 1)
df_fake_data["population_total"] = df_fake_data[sa.model_attributes.build_varlist("General", "Population")].sum(axis = 1).astype(int)

In [102]:
model_afolu.project(df_fake_data)

gdp_mmm_usd


['va_agliv_mmm_usd',
 'va_commercial_mmm_usd',
 'va_industrial_mmm_usd',
 'va_manufacturing_mmm_usd',
 'va_mining_mmm_usd']

In [66]:
sa.model_attributes.build_varlist("Land Use", ":math:\\text{CO}_2 Land Use Conversion Emission Factor")
#sa.model_attributes.dict_varreqs["category_af_lndu"].table["variable"].iloc[2]
[x for x in df_fake_data.columns if "forest" in x]

['ef_forestfires_mangroves_gg_co2_ha',
 'ef_forestfires_primary_gg_co2_ha',
 'ef_forestfires_secondary_gg_co2_ha',
 'ef_forestmethane_mangroves_gg_ch4_ha',
 'ef_forestmethane_primary_gg_ch4_ha',
 'ef_forestmethane_secondary_gg_ch4_ha',
 'ef_luconversion_croplands_to_forests_mangroves_gg_co2_ha',
 'ef_luconversion_croplands_to_forests_primary_gg_co2_ha',
 'ef_luconversion_croplands_to_forests_secondary_gg_co2_ha',
 'ef_luconversion_forests_mangroves_to_croplands_gg_co2_ha',
 'ef_luconversion_forests_mangroves_to_forests_mangroves_gg_co2_ha',
 'ef_luconversion_forests_mangroves_to_forests_primary_gg_co2_ha',
 'ef_luconversion_forests_mangroves_to_forests_secondary_gg_co2_ha',
 'ef_luconversion_forests_mangroves_to_grasslands_gg_co2_ha',
 'ef_luconversion_forests_mangroves_to_other_gg_co2_ha',
 'ef_luconversion_forests_mangroves_to_settlements_gg_co2_ha',
 'ef_luconversion_forests_mangroves_to_wetlands_gg_co2_ha',
 'ef_luconversion_forests_primary_to_croplands_gg_co2_ha',
 'ef_luconversio

In [21]:
sa.model_attributes.



UnboundLocalError: local variable 'dict_vr_vtf' referenced before assignment

In [ ]:
#
#
#
